In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import gbd_mapping as gbd
from db_queries import get_outputs, get_ids
from get_draws.api import get_draws
from vivarium import Artifact

import bep_summarizer

!whoami
!date

ndbs
Wed Feb  5 13:30:47 PST 2020


In [2]:
%load_ext autoreload
%autoreload 2

## Artifact and output directories

In [3]:
artifact_dir = '/ihme/costeffectiveness/artifacts/vivarium_gates_bep/'
results_dir = '/share/costeffectiveness/results/vivarium_gates_bep/base_model_half_day'

locations_rundates = {
    'Mali': '2020_02_04_20_04_06',
    'Tanzania': '2020_02_04_20_09_12'
}

locations = list(locations_rundates.keys())

## GBD IDs from R shiny tool

https://shiny.ihme.washington.edu/content/88/

(Can also use `get_ids`)

In [4]:
# GBD round IDs
gbd_2017 = 5

# Location IDs
tanzania = 189 # United Republic of Tanzania 
mali = 211 # Mali

# Cause IDs
all_causes = 294
diarrhea = 302 # Diarrheal diseases
lri = 322 # Lower respiratory infections
meningitis = 332
measles = 341
neonatal = 380 # Neonatal disorders
pem = 387 # Protein energy malnutrition

# Measures
deaths = 1
prevalence = 5
incidence = 6
emr = 9 # Excess mortality rate
acmr = 14 # All-cause mortality rate
csmr = 15 # Cause-specific mortality rate
relative_risk = 11
birth_prev = 38 # Birth prevalence

# Metrics
number = 1
percent = 2
rate = 3

# Age group IDs
at_birth = 164
early_nn = 2
late_nn = 3
post_nn = 4
one_to_four = 5

# Sex IDs
male=1
female=2
both=3

In [11]:
art_mali = Artifact(artifact_dir + 'mali.hdf', filter_terms=['year_start == 2017', 'age_start < 5'])

## Function to read artifact by draw, since LBWSG data was messed up

In [8]:
def read_lbwsg_data_by_draw(artifact_path, draw):
    key = 'risk_factor/low_birth_weight_and_short_gestation/exposure'
    with pd.HDFStore(artifact_path, mode='r') as store:
        index = store.get(f'{key}/index')
        draw = store.get(f'{key}/draw_{draw}')
    draw = draw.rename("value")
    data = pd.concat([index, draw], axis=1)
    return data

In [35]:
mali_lbwsg = read_lbwsg_data_by_draw(artifact_dir + 'mali.hdf', 0)
mali_lbwsg.head()

,location,sex,age_start,age_end,year_start,year_end,parameter,value
0,Mali,Female,0.0,0.019178,1990,1991,cat10,0.000000
1,Mali,Female,0.0,0.019178,1990,1991,cat106,0.001379
2,Mali,Female,0.0,0.019178,1990,1991,cat11,0.001316
3,Mali,Female,0.0,0.019178,1990,1991,cat116,0.002852
4,Mali,Female,0.0,0.019178,1990,1991,cat117,0.012758


In [36]:
mali_lbwsg.shape

(73416, 8)

In [37]:
mali_lbwsg.age_start.unique()

array([0.000000e+00, 1.917808e-02, 7.671233e-02, 1.000000e+00,
       5.000000e+00, 1.000000e+01, 1.500000e+01, 2.000000e+01,
       2.500000e+01, 3.000000e+01, 3.500000e+01, 4.000000e+01,
       4.500000e+01, 5.000000e+01, 5.500000e+01, 6.000000e+01,
       6.500000e+01, 7.000000e+01, 7.500000e+01, 8.000000e+01,
       8.500000e+01, 9.000000e+01, 9.500000e+01])

## Write function to filter data and read all draws for LBWSG

In [38]:
# Convert a list of filter terms into a pandas .query() string
filter_terms = ['year_start == 2017', 'age_start < 5', "parameter == 'cat10'"]
query_string = " and ".join(filter_terms)
query_string

"year_start == 2017 and age_start < 5 and parameter == 'cat10'"

In [39]:
mali_lbwsg_reduced = mali_lbwsg.query(query_string)
mali_lbwsg_reduced

,location,sex,age_start,age_end,year_start,year_end,parameter,value
1539,Mali,Female,0.000000,0.019178,2017,2018,cat10,0.002128
3135,Mali,Female,0.019178,0.076712,2017,2018,cat10,0.001435
4731,Mali,Female,0.076712,1.000000,2017,2018,cat10,0.000000
6327,Mali,Female,1.000000,5.000000,2017,2018,cat10,0.000000
38247,Mali,Male,0.000000,0.019178,2017,2018,cat10,0.000000
39843,Mali,Male,0.019178,0.076712,2017,2018,cat10,0.000000
41439,Mali,Male,0.076712,1.000000,2017,2018,cat10,0.000000
43035,Mali,Male,1.000000,5.000000,2017,2018,cat10,0.000000


In [44]:
index_cols = [col for col in mali_lbwsg_reduced.columns if col != 'value']
mali_lbwsg_reduced.set_index(index_cols).rename(columns={'value': 'draw_0'})

draw_0
location sex    age_start age_end  year_start year_end parameter          
Mali     Female 0.000000  0.019178 2017       2018     cat10      0.002128
                0.019178  0.076712 2017       2018     cat10      0.001435
                0.076712  1.000000 2017       2018     cat10      0.000000
                1.000000  5.000000 2017       2018     cat10      0.000000
         Male   0.000000  0.019178 2017       2018     cat10      0.000000
                0.019178  0.076712 2017       2018     cat10      0.000000
                0.076712  1.000000 2017       2018     cat10      0.000000
                1.000000  5.000000 2017       2018     cat10      0.000000

In [53]:
def read_lbwsg_data(artifact_path, filter_terms):
    query_string = " and ".join(filter_terms)
    
    data = read_lbwsg_data_by_draw(artifact_path, 0)
    data = data.query(query_string)
    index_cols = [col for col in data.columns if col != 'value']
    data = data.set_index(index_cols).rename(columns={'value': 'draw_0'})
    
    for draw in range(1,1000):
        draw_data = read_lbwsg_data_by_draw(artifact_path, draw)
        draw_data = draw_data.query(query_string)
        draw_data = draw_data.set_index(index_cols).rename(columns={'value': f'draw_{draw}'})
        data = pd.concat([data, draw_data], axis=1)
        
    return data

In [54]:
mali_lbwsg = read_lbwsg_data(artifact_dir + 'mali.hdf', ['year_start == 2017', 'age_start < 5'])
mali_lbwsg.head()

draw_0  \
location sex    age_start age_end  year_start year_end parameter             
Mali     Female 0.0       0.019178 2017       2018     cat10      0.002128   
                                                       cat106     0.001562   
                                                       cat11      0.001454   
                                                       cat116     0.003774   
                                                       cat117     0.014307   

                                                                    draw_1  \
location sex    age_start age_end  year_start year_end parameter             
Mali     Female 0.0       0.019178 2017       2018     cat10      0.001768   
                                                       cat106     0.001507   
                                                       cat11      0.001369   
                                                       cat116     0.003472   
                                                       cat117     0.014481   

                                                                    draw_2  \
location sex    age_start age_end  year_start year_end parameter             
Mali     Female 0.0       0.019178 2017       2018     cat10      0.001924   
                                                       cat106     0.001917   
                                                       cat11      0.001351   
                                                       cat116     0.003759   
                                                       cat117     0.013703   

                                                                    draw_3  \
location sex    age_start age_end  year_start year_end parameter             
Mali     Female 0.0       0.019178 2017       2018     cat10      0.002649   
                                                       cat106     0.001878   
                                                       cat11      0.001396   
                                                       cat116     0.004055   
                                                       cat117     0.013855   

                                                                    draw_4  \
location sex    age_start age_end  year_start year_end parameter             
Mali     Female 0.0       0.019178 2017       2018     cat10      0.001953   
                                                       cat106     0.002094   
                                                       cat11      0.001431   
                                                       cat116     0.004935   
                                                       cat117     0.016841   

                                                                    draw_5  \
location sex    age_start age_end  year_start year_end parameter             
Mali     Female 0.0       0.019178 2017       2018     cat10      0.002106   
                                                       cat106     0.002062   
                                                       cat11      0.001580   
                                                       cat116     0.004220   
                                                       cat117     0.014736   

                                                                    draw_6  \
location sex    age_start age_end  year_start year_end parameter             
Mali     Female 0.0       0.019178 2017       2018     cat10      0.002140   
                                                       cat106     0.001822   
                                                       cat11      0.001661   
                                                       cat116     0.004160   
                                                       cat117     0.015148   

                                                                    draw_7  \
location sex    age_start age_end  year_start year_end parameter             
Mali     Female 0.0       0.019178 2017       2018     cat10      0.001769   
     

In [55]:
mali_lbwsg.shape

(456, 1000)

### Woo hoo! Looks like the function works

let's figure out what's in the index:

456 rows = (57 categories) x (4 age groups) x (2 sexes)

In [57]:
# Count the total number of LBWSG categories: 57
mali_lbwsg.index.get_level_values('parameter').nunique()

57

In [58]:
456/57 # 8 = (4 age groups) x (2 sexes)

8.0